In [1]:
import os
import pandas as pd
import numpy as np
import cv2
import torch
from torch.utils.data import Dataset, DataLoader
from torchvision import transforms
import torch.nn as nn
import torchvision.models as models
from PIL import Image, ImageDraw
import random
from collections import defaultdict

In [2]:
class RoadSignDataset(Dataset):
    def __init__(self, images_dir, labels_dir, transform=None, unknown_transform=None, unknown_crop_size=(224, 224), unknown_samples_per_image=1, is_validation=False):
        self.images_dir = images_dir
        self.labels_dir = labels_dir
        self.transform = transform
        self.unknown_transform = unknown_transform
        self.unknown_crop_size = unknown_crop_size
        self.unknown_samples_per_image = unknown_samples_per_image
        self.is_validation = is_validation
        self.images = [f for f in os.listdir(images_dir) if f.endswith('.jpg')]
        self.annotations = self._load_annotations()
        print(f"Loaded {len(self.images)} images with annotations")

    def _load_annotations(self):
        annotations = {}
        for img_file in self.images:
            img_name = os.path.splitext(img_file)[0]
            label_file = os.path.join(self.labels_dir, img_name + '.csv')
            try:
                df = pd.read_csv(label_file, header=None)
                if df.empty:
                    annotations[img_file] = "unknown"
                else:
                    df = df[~df[4].str.lower().isin(['ff'])]
                    annotations[img_file] = df.values.tolist()
                print(f"Processed {label_file}, found {len(annotations[img_file]) if annotations[img_file] != 'unknown' else 'unknown'} annotations")
            except pd.errors.EmptyDataError:
                print(f"EmptyDataError: {label_file}")
                annotations[img_file] = "unknown"
            except Exception as e:
                print(f"Error processing {label_file}: {e}")
                annotations[img_file] = "unknown"
        return annotations

    def _get_unknown_crops(self, image, label_boxes):
        if self.is_validation:
            return []
        width, height = image.size
        crops = []
        if width < self.unknown_crop_size[0] or height < self.unknown_crop_size[1]:
            print(f"Skipping unknown crop for image of size ({width}, {height})")
            return crops
        for _ in range(self.unknown_samples_per_image):
            attempts = 0
            while attempts < 10:
                left = random.randint(0, width - self.unknown_crop_size[0])
                top = random.randint(0, height - self.unknown_crop_size[1])
                right = left + self.unknown_crop_size[0]
                bottom = top + self.unknown_crop_size[1]
                crop_box = (left, top, right, bottom)
                if not any(self._boxes_intersect(crop_box, box) for box in label_boxes):
                    crop = image.crop(crop_box)
                    if self.unknown_transform:
                        crop = self.unknown_transform(crop)
                    crops.append(crop)
                    break
                attempts += 1
        return crops

    def _boxes_intersect(self, boxA, boxB):
        leftA, topA, rightA, bottomA = boxA
        leftB, topB, rightB, bottomB = boxB
        return not (leftA >= rightB or rightA <= leftB or topA >= bottomB or bottomA <= topB)

    def __len__(self):
        return len(self.images)

    def __getitem__(self, idx):
        img_file = self.images[idx]
        img_path = os.path.join(self.images_dir, img_file)
        image = Image.open(img_path).convert('RGB')
        #print(f"Loading image: {img_path}")

        labels = self.annotations[img_file]
        crops = []
        categories = []

        if labels == "unknown":
            if self.unknown_transform:
                image = self.unknown_transform(image)
            else:
                image = self.transform(image) if self.transform else image
            crops.append(image)
            categories.append("unknown")
        else:
            label_boxes = []
            for label in labels:
                if len(label) == 5:
                    x1, y1, x2, y2, category = label
                    label_boxes.append((x1, y1, x2, y2))
                    crop = image.crop((x1, y1, x2, y2))
                    if self.transform:
                        crop = self.transform(crop)
                    crops.append(crop)
                    categories.append(category)
                else:
                    print(f"Skipping invalid label: {label}")

            unknown_crops = self._get_unknown_crops(image, label_boxes)
            crops.extend(unknown_crops)
            categories.extend(["unknown"] * len(unknown_crops))

        return crops, categories



def custom_collate_fn(batch):
    crops = [item[0] for item in batch]
    categories = [item[1] for item in batch]
    crops = [crop for sublist in crops for crop in sublist]
    categories = [category for sublist in categories for category in sublist]
    return crops, categories

#calculer le nombre des labels dans un dataset
def count_labels_in_dataset(dataset):
    label_counts = defaultdict(int)
    for i in range(len(dataset)):
        _, categories = dataset[i]
        for category in categories:
            label_counts[category] += 1
    return label_counts

In [3]:
images_dir = './train/images'
labels_dir = './train/labels'
val_images_dir = './val/images'
val_labels_dir = './val/labels'

transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
])

unknown_transform = transforms.Compose([
    transforms.RandomHorizontalFlip(),
    transforms.RandomVerticalFlip(),
    transforms.ColorJitter(brightness=0.5, contrast=0.5, saturation=0.5, hue=0.5),
    transforms.RandomRotation(degrees=15),
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
])
#training
dataset = RoadSignDataset(images_dir, labels_dir, transform=transform, unknown_transform=unknown_transform, unknown_samples_per_image=1,is_validation=False)
dataloader = DataLoader(dataset, batch_size=32, shuffle=True, collate_fn=custom_collate_fn)
#validation
val_dataset = RoadSignDataset(val_images_dir, val_labels_dir, transform=transform, unknown_transform=unknown_transform, unknown_samples_per_image=1,is_validation=True)
val_dataloader = DataLoader(val_dataset, batch_size=32, shuffle=False, collate_fn=custom_collate_fn)

for batch in dataloader:
    crops, categories = batch
    for crop, category in zip(crops, categories):
        print(crop.shape, category)

Processed ./train/labels\0001.csv, found 2 annotations
Processed ./train/labels\0002.csv, found 1 annotations
Processed ./train/labels\0003.csv, found 4 annotations
Processed ./train/labels\0005.csv, found 3 annotations
Processed ./train/labels\0006.csv, found 2 annotations
Processed ./train/labels\0007.csv, found 3 annotations
Processed ./train/labels\0008.csv, found 4 annotations
Processed ./train/labels\0009.csv, found 4 annotations
EmptyDataError: ./train/labels\0010.csv
Processed ./train/labels\0011.csv, found 1 annotations
Processed ./train/labels\0012.csv, found 1 annotations
Processed ./train/labels\0013.csv, found 1 annotations
Processed ./train/labels\0014.csv, found 1 annotations
Processed ./train/labels\0015.csv, found 2 annotations
EmptyDataError: ./train/labels\0016.csv
Processed ./train/labels\0017.csv, found 3 annotations
EmptyDataError: ./train/labels\0018.csv
Processed ./train/labels\0019.csv, found 1 annotations
Processed ./train/labels\0020.csv, found 2 annotations


In [4]:
# calculer le nombre des images de chaque label dans training set
label_counts = count_labels_in_dataset(dataset)
print("Label counts in traingset:")
for label, count in label_counts.items():
    print(f"{label}: {count}")

Skipping unknown crop for image of size (133, 110)
Label counts in traingset:
frouge: 79
unknown: 676
ceder: 120
interdiction: 288
fvert: 94
stop: 98
danger: 155
obligation: 106
forange: 56


In [5]:
label_counts = count_labels_in_dataset(val_dataset)
print("Label counts in validation dataset:")
for label, count in label_counts.items():
    print(f"{label}: {count}")

Label counts in validation dataset:
frouge: 13
interdiction: 33
ceder: 16
obligation: 14
stop: 14
danger: 17
forange: 5
unknown: 6
fvert: 2


In [6]:
def initialize_model(num_classes, use_pretrained=True):
    model = models.resnet18(weights=models.ResNet18_Weights.DEFAULT if use_pretrained else None)
    num_ftrs = model.fc.in_features
    model.fc = nn.Linear(num_ftrs, num_classes)
    return model

def train_model(model, criterion, optimizer, dataloader, num_epochs=25):
    for epoch in range(num_epochs):
        model.train()
        running_loss = 0.0
        for inputs, labels in dataloader:
            inputs = torch.stack(inputs).to(device)
            labels = torch.tensor([label_to_idx[label] for label in labels]).to(device)
            
            optimizer.zero_grad()
            outputs = model(inputs)
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()
            
            running_loss += loss.item() * inputs.size(0)
        
        epoch_loss = running_loss / len(dataloader.dataset)
        print(f'Epoch {epoch}/{num_epochs - 1}, Loss: {epoch_loss:.4f}')

    print('Training complete')

# évaluer le modèle
def evaluate_model(model, dataloader):
    model.eval()
    correct = 0
    total = 0
    num_images = 0
    with torch.no_grad():
        for inputs, labels in dataloader:
            inputs = torch.stack(inputs).to(device)
            labels = torch.tensor([label_to_idx[label] for label in labels]).to(device)
            outputs = model(inputs)
            _, predicted = torch.max(outputs, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()
            num_images += len(inputs)
            
    accuracy = 100 * correct / total if total > 0 else None
    print(f'Accuracy: {accuracy:.2f}%' if accuracy is not None else "Accuracy calculation failed")
    print(f'Total images validated: {num_images}')
    return accuracy


#définir les labels
label_to_idx = {label: idx for idx, label in enumerate(["danger", "interdiction", "obligation", "stop", "ceder", "frouge", "forange", "fvert", "unknown"])}
idx_to_label = {idx: label for label, idx in label_to_idx.items()}

# initialiser le modèle
num_classes = 9
model = initialize_model(num_classes, use_pretrained=False)
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
model = model.to(device)
#définir la fonction de perte et l'optimiseur
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=0.001)

train_model(model, criterion, optimizer, dataloader)

Skipping unknown crop for image of size (133, 110)
Epoch 0/24, Loss: 3.5043
Skipping unknown crop for image of size (133, 110)
Epoch 1/24, Loss: 2.1733
Skipping unknown crop for image of size (133, 110)
Epoch 2/24, Loss: 1.7960
Skipping unknown crop for image of size (133, 110)
Epoch 3/24, Loss: 1.3062
Skipping unknown crop for image of size (133, 110)
Epoch 4/24, Loss: 1.3224
Skipping unknown crop for image of size (133, 110)
Epoch 5/24, Loss: 1.0061
Skipping unknown crop for image of size (133, 110)
Epoch 6/24, Loss: 0.9116
Skipping unknown crop for image of size (133, 110)
Epoch 7/24, Loss: 0.7694
Skipping unknown crop for image of size (133, 110)
Epoch 8/24, Loss: 0.6542
Skipping unknown crop for image of size (133, 110)
Epoch 9/24, Loss: 0.5402
Skipping unknown crop for image of size (133, 110)
Epoch 10/24, Loss: 0.4695
Skipping unknown crop for image of size (133, 110)
Epoch 11/24, Loss: 0.4545
Skipping unknown crop for image of size (133, 110)
Epoch 12/24, Loss: 0.4063
Skipping 

In [7]:
torch.save(model.state_dict(), 'resnet18_road_sign_model.pth')

In [8]:
#vgg16
def initialize_vgg16_model(num_classes, use_pretrained=True):
    model = models.vgg16(weights=models.VGG16_Weights.DEFAULT if use_pretrained else None)
    num_ftrs = model.classifier[6].in_features
    model.classifier[6] = nn.Linear(num_ftrs, num_classes)
    return model

# initializer le modèle VGG16
model_vgg16 = initialize_vgg16_model(num_classes, use_pretrained=False)
model_vgg16 = model_vgg16.to(device)

# entraîner le modèle vgg16
train_model(model_vgg16, criterion, optimizer, dataloader)

torch.save(model_vgg16.state_dict(), 'vgg16_road_sign_model.pth')

Skipping unknown crop for image of size (133, 110)
Epoch 0/24, Loss: 5.2234
Skipping unknown crop for image of size (133, 110)
Epoch 1/24, Loss: 5.2160
Skipping unknown crop for image of size (133, 110)
Epoch 2/24, Loss: 5.2262
Skipping unknown crop for image of size (133, 110)
Epoch 3/24, Loss: 5.2304
Skipping unknown crop for image of size (133, 110)
Epoch 4/24, Loss: 5.2174
Skipping unknown crop for image of size (133, 110)
Epoch 5/24, Loss: 5.2205
Skipping unknown crop for image of size (133, 110)
Epoch 6/24, Loss: 5.2238
Skipping unknown crop for image of size (133, 110)
Epoch 7/24, Loss: 5.2265
Skipping unknown crop for image of size (133, 110)
Epoch 8/24, Loss: 5.2238
Skipping unknown crop for image of size (133, 110)
Epoch 9/24, Loss: 5.2187
Skipping unknown crop for image of size (133, 110)
Epoch 10/24, Loss: 5.2111
Skipping unknown crop for image of size (133, 110)
Epoch 11/24, Loss: 5.2196
Skipping unknown crop for image of size (133, 110)
Epoch 12/24, Loss: 5.2288
Skipping 

In [9]:
def initialize_densenet_model(num_classes, use_pretrained=True):
    model = models.densenet121(weights=models.DenseNet121_Weights.DEFAULT if use_pretrained else None)
    num_ftrs = model.classifier.in_features
    model.classifier = nn.Linear(num_ftrs, num_classes)
    return model

model_densenet = initialize_densenet_model(num_classes, use_pretrained=False)
model_densenet = model_densenet.to(device)

train_model(model_densenet, criterion, optimizer, dataloader)

torch.save(model_densenet.state_dict(), 'densenet_road_sign_model.pth')

Skipping unknown crop for image of size (133, 110)
Epoch 0/24, Loss: 5.2255
Skipping unknown crop for image of size (133, 110)
Epoch 1/24, Loss: 5.2013
Skipping unknown crop for image of size (133, 110)
Epoch 2/24, Loss: 5.2218
Skipping unknown crop for image of size (133, 110)
Epoch 3/24, Loss: 5.2155
Skipping unknown crop for image of size (133, 110)
Epoch 4/24, Loss: 5.1872
Skipping unknown crop for image of size (133, 110)
Epoch 5/24, Loss: 5.2037
Skipping unknown crop for image of size (133, 110)
Epoch 6/24, Loss: 5.2225
Skipping unknown crop for image of size (133, 110)
Epoch 7/24, Loss: 5.2215
Skipping unknown crop for image of size (133, 110)
Epoch 8/24, Loss: 5.1911
Skipping unknown crop for image of size (133, 110)
Epoch 9/24, Loss: 5.2135
Skipping unknown crop for image of size (133, 110)
Epoch 10/24, Loss: 5.2131
Skipping unknown crop for image of size (133, 110)
Epoch 11/24, Loss: 5.2145
Skipping unknown crop for image of size (133, 110)
Epoch 12/24, Loss: 5.2086
Skipping 

In [10]:
def initialize_resnet18_model(num_classes, use_pretrained=False):
    model = models.resnet18(weights=models.ResNet18_Weights.DEFAULT if use_pretrained else None)
    num_ftrs = model.fc.in_features
    model.fc = nn.Linear(num_ftrs, num_classes)
    return model

def initialize_vgg16_model(num_classes, use_pretrained=False):
    model = models.vgg16(weights=models.VGG16_Weights.DEFAULT if use_pretrained else None)
    num_ftrs = model.classifier[6].in_features
    model.classifier[6] = nn.Linear(num_ftrs, num_classes)
    return model

def initialize_densenet_model(num_classes, use_pretrained=False):
    model = models.densenet121(weights=models.DenseNet121_Weights.DEFAULT if use_pretrained else None)
    num_ftrs = model.classifier.in_features
    model.classifier = nn.Linear(num_ftrs, num_classes)
    return model
model_initializers = {
    'resnet18': initialize_resnet18_model,
    'vgg16': initialize_vgg16_model,
    'densenet': initialize_densenet_model,
}
results = {}
# model_names = ['resnet18','vgg16', 'inception', 'densenet']
for model_name, initialize_model in model_initializers.items():
    model = initialize_model(num_classes, use_pretrained=False)
    model.load_state_dict(torch.load(f'{model_name}_road_sign_model.pth'))
    model = model.to(device)
    print(f'Evaluating {model_name} model...')
    accuracy = evaluate_model(model, val_dataloader)
    results[model_name] = accuracy

Evaluating resnet18 model...
Accuracy: 92.50%
Total images validated: 120
Evaluating vgg16 model...
Accuracy: 1.67%
Total images validated: 120
Evaluating densenet model...
Accuracy: 8.33%
Total images validated: 120
